In [ ]:
from pipeline import PrepPipe, OffensiveFormation, DefensiveClustering

In [186]:
prep_pipe = PrepPipe()

positional data already downloaded.
reading positional data.
returning positional data.


In [256]:
X_train, X_test, y_train, y_test = prep_pipe.clean_data()
y_train_x = y_train.iloc[:, 0]
y_yrain_y = y_train.iloc[:, 1]
y_test_x = y_test.iloc[:, 0]
y_test_y = y_test.iloc[:, 1]

In [188]:
off_col = prep_pipe.train_test.ofc.drop(['gameId', 'playId', 'gamePlayId', 'week'], axis=1).columns

In [189]:
def_col = prep_pipe.train_test.dfc.drop(['week', 'index'], axis=1).columns

In [190]:
off_form_cols = off_col[:-9]
off_info_cols = off_col[-9:]

In [273]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd

off_form_pipe = Pipeline([('off_form_pred', OffensiveFormation())])
off_info_pipe = Pipeline([('scaler', StandardScaler())])
off_pre_one_pipe = ColumnTransformer([('info', off_info_pipe, off_info_cols),
                                      ('form', off_form_pipe, off_form_cols),
                                   ])
off_pre_one_add_col = Pipeline([('off_pre_one', off_pre_one_pipe),
                               ('func_trans', FunctionTransformer(lambda x: pd.DataFrame(x,
                                columns=list(off_info_cols) + list(off_form_cols))))])
form_one_pipe = ColumnTransformer([('off_form_one', OneHotEncoder(), [-1]),
                                   ('nothing', FunctionTransformer(lambda x: x), slice(-1))])
off_full_pipe = Pipeline([('full_cols', off_pre_one_add_col), ('one_hot', form_one_pipe)])

def_cluster_pipe = Pipeline([('def_clust', DefensiveClustering())])
form_one_pipe = ColumnTransformer([('def_clust_one', OneHotEncoder(), [-1]),
                                   ('nothing', FunctionTransformer(lambda x: x), slice(-1))])
def_full_pipe = Pipeline([('def_clust_full', def_cluster_pipe), ('def_clust_one', form_one_pipe)])

full_pipe = ColumnTransformer([('off', off_full_pipe, off_col), ('def', def_full_pipe, def_col)])
full_pipe_model = Pipeline([('full_pipe', full_pipe), ('model', LogisticRegression())])

In [310]:
X_scaled = full_pipe.fit_transform(X_test)
X_scaled_off = off_full_pipe.fit_transform(X_test[off_col])

Offensive formation model fitted
Offensive formation predicted
KMeans and PCA fitted
Defensive position transformed
Offensive formation model fitted
Offensive formation predicted


In [337]:
model1 = LogisticRegression()

In [340]:
model1.fit(X_scaled, y_test_x)

LogisticRegression()

In [342]:
y_pred = model1.predict(X_scaled)

In [343]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test_x, y_pred)

array([[ 91, 385,  66,   9],
       [ 60, 993, 140,  20],
       [ 42, 540, 215,   9],
       [ 25, 357,  82,  28]])